In [3]:
import os

os.environ['TORCH_HOME'] = "/data/amathur-23/ROB313/"

In [16]:
from models import ResNetBinaryClassifier
import torch 

projection_path = '/data/amathur-23/ROB313/models/best_model.pth'

model = ResNetBinaryClassifier()
model = torch.load(projection_path)
# model.load_state_dict(torch.load(projection_path))


Using ResNet50 with 2048 FC input features


In [19]:
model = ResNetBinaryClassifier(backbone='resnet101')
model.load_state_dict(torch.load(projection_path))

Using ResNet50 with 2048 FC input features


<All keys matched successfully>

In [22]:
from utils.datasets import WildfireDataset

dataset = WildfireDataset('/data/amathur-23/ROB313', split='test')

loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

Loading meta file: /data/amathur-23/ROB313/test.csv


In [23]:
import numpy as np

import matplotlib.pyplot as plt

# Set the model to evaluation mode
model.eval()

# Function to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Iterate over the data in the loader
for images, labels in loader:
    outputs = model(images)
    preds = torch.round(torch.sigmoid(outputs)).squeeze()

    # Find indices where prediction does not match the label
    mismatch_indices = (preds != labels).nonzero(as_tuple=True)[0]

    # Show images where prediction does not match the label
    for idx in mismatch_indices:
        imshow(images[idx])
        print(f"Prediction: {preds[idx].item()}, Label: {labels[idx].item()}")

ValueError: too many values to unpack (expected 2)